In [1]:
import os
import threading
import pandas as pd
import random, time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
px = pd.read_csv('proxies.csv')
px.sample()

,alive,alive_since,anonymity,average_timeout,first_seen,ip_data_as,ip_data_asname,ip_data_city,ip_data_continent,ip_data_continentCode,...,last_seen,port,protocol,proxy,ssl,timeout,times_alive,times_dead,uptime,ip
630,True,1.738096e+09,elite,10178.115991,1.720147e+09,AS12876 SCALEWAY S.A.S.,AS12876,Paris,Europe,EU,...,1.738096e+09,58714,socks4,socks4://212.83.143.151:58714,True,10775.727034,395,74009,0.530885,212.83.143.151


In [3]:
proxies = px[px['alive'] == True]["proxy"].to_list()

In [4]:
profiles = [
    {"name": "Akshat", "profile_path": r"C:\Users\jaksh\AppData\Local\Google\Chrome\User Data", "id": 1},
    {"name":"Sarcaxit", "profile_path": r"C:\Users\jaksh\AppData\Local\Google\Chrome\User Data", "id": 6},
    {"name":"Ishika", "profile_path": r"C:\Users\jaksh\AppData\Local\Google\Chrome\User Data", "id": 12},
    {"name":"Mummy", "profile_path": r"C:\Users\jaksh\AppData\Local\Google\Chrome\User Data", "id": 13}
]

In [13]:
search_query = "random topics"

In [6]:
import requests
import threading

# Shared list to store working proxies
working_proxies = []

# Lock for thread-safe access to shared list
lock = threading.Lock()

# Function to test if a proxy works
def check_proxy(proxy):
    try:
        # Define the proxies dictionary
        proxies_dict = {
            "http": proxy,
            "https": proxy,
        }
        # Make a request to httpbin to check the IP
        response = requests.get('https://httpbin.org/ip', proxies=proxies_dict, timeout=5)
        if response.status_code == 200:
            print(f"Proxy works: {proxy}")
            with lock:  # Use lock to safely append to the shared list
                working_proxies.append(proxy)
    except Exception as e:
        pass

# Create and start threads for each proxy
threads = []
for proxy in proxies:
    thread = threading.Thread(target=check_proxy, args=(proxy,))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Print the list of working proxies
print("\nWorking Proxies:")
print(working_proxies)


Proxy works: http://113.23.155.110:1231
Proxy works: http://83.217.23.34:8090
Proxy works: http://18.159.113.250:3128
Proxy works: http://36.111.142.2:12798
Proxy works: http://15.168.228.9:3128
Proxy works: http://58.23.152.29:7080
Proxy works: http://36.111.142.177:12792
Proxy works: http://106.120.183.118:12700
Proxy works: http://98.8.195.160:443
Proxy works: http://39.185.44.125:5911
Proxy works: http://136.243.82.121:1083
Proxy works: http://103.226.251.105:46273
Proxy works: http://115.72.13.202:10001
Proxy works: http://36.111.142.177:12700
Proxy works: http://157.20.252.221:8080
Proxy works: http://59.39.63.54:6399
Proxy works: http://111.1.61.49:3128
Proxy works: http://115.72.8.252:10001
Proxy works: http://111.177.48.18:9502
Proxy works: http://189.240.60.172:9090
Proxy works: http://102.223.186.246:8888
Proxy works: http://39.185.67.193:5911
Proxy works: http://62.33.53.248:3128
Proxy works: http://39.185.41.193:5911
Proxy works: http://51.210.54.186:8585
Proxy works: http

In [14]:
def perform_random_actions(driver):
    actions = [
        "scroll",  # Scroll the page
        "random_click",  # Click on a random link
        "go_back",  # Go back to the previous page
    ]

    action = random.choice(actions)
    print(f"Performing action: {action}")

    try:
        if action == "scroll":
            # Randomly scroll up or down
            scroll_distance = random.randint(200, 1000)
            driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
            time.sleep(random.uniform(2, 5))

        elif action == "stay_idle":
            # Do nothing for a few seconds
            time.sleep(random.uniform(3, 6))

        elif action == "random_click":
            # Click on a random link from search results
            links = driver.find_elements(By.XPATH, "//a")
            if links:
                random_link = random.choice(links)
                random_link.click()
                time.sleep(random.uniform(3, 6))
                driver.back()

        elif action == "go_back":
            # Go back to the previous page
            driver.back()
            time.sleep(random.uniform(2, 4))

    except Exception as e:
        print(f"Error during action {action}: {e}")


In [8]:
profile = profiles[2]

In [15]:
def test(i=1):
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-extensions")
    options.add_argument(f"--user-data-dir={profile['profile_path']}") 
    options.add_argument(f"--profile-directory=Profile {profile['id']}") 

    if i==5:
        return

    try:
        options.add_argument(f"--proxy-server={working_proxies[i]}")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        # Open Bing and perform the search
        driver.get("https://www.bing.com/")
        time.sleep(10)  # Allow page to load

        # Find the search box, enter the query, and submit the search
        search_box = driver.find_element(By.CLASS_NAME, "sb_form_q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(4)  # Wait for results to load
        perform_random_actions(driver)
        time.sleep(5)
        
    except:
        test(i+1)

    finally:
        # Close the browser
        driver.quit()

In [16]:
test()

Performing action: go_back


In [ ]:
def run_searches():
    threads = []
    for proxy in proxies:
        thread = threading.Thread(target=test, args=(proxy,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

run_searches()

In [ ]:
def perform_search(profile):
    # Set Chrome options to use the given profile and proxy
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-extensions")
    options.add_argument(f"user-data-dir={profile['profile_path']}")  # Specify user profile path
    
    # Configure proxy settings
    if "proxy" in profile and profile["proxy"]:
        options.add_argument(f"--proxy-server=http://{profile['proxy']}")

    # Launch the browser with the given profile and proxy
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        # Open Bing and perform the search
        driver.get("https://www.bing.com/")
        time.sleep(2)  # Allow page to load

        # Find the search box, enter the query, and submit the search
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(5)  # Wait for results to load
        
    finally:
        # Close the browser
        driver.quit()

In [1]:
import pandas as pd

proxies = pd.read_csv('proxies.csv')
proxies.sample()

,alive,alive_since,anonymity,average_timeout,first_seen,ip_data_as,ip_data_asname,ip_data_city,ip_data_continent,ip_data_continentCode,...,last_seen,port,protocol,proxy,ssl,timeout,times_alive,times_dead,uptime,ip
1019,True,1.737515e+09,elite,4651.403947,1.736975e+09,AS36666 GloboTech Communications,GTCOMM,Montreal,North America,NaN,...,1.737515e+09,32709,socks4,socks4://67.43.236.18:32709,True,891.138792,11,3319,0.33033,67.43.236.18


In [3]:
# Filter the proxies for best candidates
filtered_proxies = proxies[
    (proxies['alive'] == True) &
    (proxies['timeout'] <= 3000) &  # Keep proxies with low timeout
    (proxies['anonymity'].str.contains('elite|high', case=False)) &  # High anonymity
    (proxies['ssl'] == True)  # Ensure SSL support
]


In [7]:
filtered_proxies[filtered_proxies["ip_data_country"] == "India"]

,alive,alive_since,anonymity,average_timeout,first_seen,ip_data_as,ip_data_asname,ip_data_city,ip_data_continent,ip_data_continentCode,...,last_seen,port,protocol,proxy,ssl,timeout,times_alive,times_dead,uptime,ip
201,True,1.738160e+09,elite,3463.150186,1.697037e+09,AS134260 ZERONE Networks pvt ltd,ZERONE-AS-IN,Perambalur,Asia,AS,...,1.738160e+09,5678,socks4,socks4://103.197.32.205:5678,True,1885.040998,19151,290443,6.185843,103.197.32.205
925,True,1.737919e+09,elite,5336.330525,1.710693e+09,AS136371 Airwir Technologies Private Limited,AIRWIR-AS,Bengaluru,Asia,AS,...,1.737919e+09,4145,socks4,socks4://103.86.1.22:4145,True,744.435549,3306,198363,1.639320,103.86.1.22
995,True,1.737631e+09,elite,4477.982647,1.710818e+09,AS133685 SDN TELECOM PVT LTD,SDNTELE-AS,Vijayawada,Asia,AS,...,1.737631e+09,4145,socks4,socks4://103.44.15.193:4145,True,1408.928394,3407,163125,2.045853,103.44.15.193


In [9]:
import requests, threading

# Test function
def test_proxy(proxy_ip, proxy_port, proxy_protocol):
    proxies = {
        "http": f"{proxy_protocol}://{proxy_ip}:{proxy_port}",
        "https": f"{proxy_protocol}://{proxy_ip}:{proxy_port}"
    }
    try:
        response = requests.get("https://www.bing.com", proxies=proxies, timeout=5)
        if response.status_code == 200:
            print(f"Proxy works: {proxy_ip}:{proxy_port}")
            return True
        else:
            print(f"Failed: {proxy_ip}:{proxy_port} - Status Code: {response.status_code}")
            return False
    except Exception as e:
        print(f"Failed: {proxy_ip}:{proxy_port} - Error: {e}")
        return False

# Iterate over filtered proxies
for _, row in filtered_proxies.iterrows():
    proxy_ip = row['ip']
    proxy_port = row['port']
    proxy_protocol = row['protocol']
    thread = threading.Thread(target=test_proxy, args=(proxy_ip, proxy_port, proxy_protocol))
    thread.start()

# Wait for all threads to complete
for thread in threading.enumerate():
    thread.join()

Failed: 103.197.32.205:5678 - Error: SOCKSHTTPSConnectionPool(host='www.bing.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x000001EFF6A2AED0>: Failed to establish a new connection: Connection closed unexpectedly'))


RuntimeError: cannot join current thread

Failed: 223.255.159.202:7777 - Error: SOCKSHTTPSConnectionPool(host='www.bing.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x000001EFF6CD1510>: Failed to establish a new connection: [WinError 10054] An existing connection was forcibly closed by the remote host'))
Failed: 45.204.207.128:80 - Error: SOCKSHTTPSConnectionPool(host='www.bing.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x000001EFF6A3B410>: Failed to establish a new connection: SOCKS4 proxy server sent invalid data'))
Failed: 51.75.126.150:50111 - Error: SOCKSHTTPSConnectionPool(host='www.bing.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x000001EFF6D56850>: Failed to establish a new connection: [WinError 10054] An existing connection was forcibly closed by th